<a href="https://colab.research.google.com/github/TiagoBarantini/Pro_Eng_dados/blob/main/prepara%C3%A7%C3%A3o_orquestra%C3%A7%C3%A3o_e_fluxos_de_dados_aula04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
#instalação JAVA
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

#instalação do pyspark
pip install -q pyspark

#download dos dados utilizados
mkdir titanic
curl https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv -o titanic/titanic.csv

In [2]:
#definir uma variavel de ambiente para identificador local do JAVA
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
#Definir a sessão spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [4]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
#leitura dos dados
titanic = (
    spark
    .read
    .csv("titanic", header=True, sep=";", inferSchema=True)
  )

titanic.printSchema()

## perguntas
1 - quantos passageiros sobreviveram ao naufrágio do Titanic?
2 - Quantos Sobreviveram entre Homens e mulheres?
3 - quantos sobreviveram entre pessoas que viajaram em classes diferentes?
4 - Quantas pessoas sobreviveram por sexo, classe de viagem e que tinha idade maior do que a mé dia de todo navio

In [12]:
#pergunta 1
titanic.where('Survived = 1').count()

342

In [9]:
#registrat o dataset como uma view
titanic.createOrReplaceTempView('titanicSQL')

In [11]:
#pergunta 1 com sql
spark.sql("""
  SELECT
    COUNT(1) as Contagem
  FROM titanicSQL
  WHERE Survived = 1

""").show()

+--------+
|Contagem|
+--------+
|     342|
+--------+



In [15]:
#pergunta 2
(
    titanic
    .where('Survived = 1')
    .groupBy('Sex')
    .count()
    .show()
)

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



In [16]:
#pergunta 2
spark.sql("""
  SELECT
    Sex,
    COUNT(1) as Contagem
  FROM titanicSQL
  WHERE Survived = 1
  GROUP BY Sex
""").show()

+------+--------+
|   Sex|Contagem|
+------+--------+
|female|     233|
|  male|     109|
+------+--------+



In [18]:
#pergunta 3
(
    titanic
    .where('Survived = 1')
    .groupBy('Pclass')
    .count()
    .show()
)

+------+-----+
|Pclass|count|
+------+-----+
|     1|  136|
|     3|  119|
|     2|   87|
+------+-----+



In [19]:
#pergunta 3
spark.sql("""
  SELECT
    Pclass,
    COUNT(1) as Contagem
  FROM titanicSQL
  WHERE Survived = 1
  GROUP BY Pclass
""").show()

+------+--------+
|Pclass|Contagem|
+------+--------+
|     1|     136|
|     3|     119|
|     2|      87|
+------+--------+



In [21]:
#pergunta 4
spark.sql("""
  SELECT
    Sex, Pclass,
    COUNT(1) as Contagem
  FROM titanicSQL
  WHERE Survived = 1
      AND Age > (SELECT AVG(Age) FROM titanicSQL)
  GROUP BY Sex, Pclass
  ORDER BY Sex, Pclass
""").show()

+------+------+--------+
|   Sex|Pclass|Contagem|
+------+------+--------+
|female|     1|      54|
|female|     2|      30|
|female|     3|       8|
|  male|     1|      27|
|  male|     2|       5|
|  male|     3|      10|
+------+------+--------+



In [33]:
#pergunta 4

(
    titanic
    .where('Survived = 1')
    .where(f.col("Age") > 26.7)
    .groupBy('Sex','Pclass')
    .count()
    .show()
)

+------+------+-----+
|   Sex|Pclass|count|
+------+------+-----+
|  male|     3|   16|
|female|     3|   12|
|female|     1|   55|
|female|     2|   40|
|  male|     2|    5|
|  male|     1|   32|
+------+------+-----+

